# Active learning infrastructure objects

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/baal-org/baal/blob/master/notebooks/fundamentals/active-learning.ipynb)

Active learning, or interactively choosing datapoints to request labels for,
presents a challenge that requires some data handling infrastructure that's
slightly different to the normal pytorch dataset classes. In particular, a
dataset is no longer a static thing, but instead grows as you progress through
your experiment or your application.

To handle these needs, `baal` contains:

- `ActiveLearningDataset`, which is a pytorch dataset that lets you
  interactively label data.
- `ActiveLearningLoop`, which wraps an `ActiveLearningDataset` and simplifies
  experiments by allowing you to call `step` whenever you want to label data.

The `ActiveLearningDataset` wraps another pytorch dataset. For an example on
how to use it, you can take a look at how we turn the MNIST dataset into an
active learning dataset:


In [2]:
path = "/tmp"

In [4]:
from torchvision import transforms, datasets
from baal.active.dataset import ActiveLearningDataset

transform = transforms.Compose([transforms.Grayscale(3), transforms.ToTensor()])
test_transform = transform

active_mnist = ActiveLearningDataset(
    datasets.MNIST(path, train=True, transform=transform),
    pool_specifics={'transform': test_transform},
)


As you can see, this is a fairly thin wrapper around MNIST. But, we can now
check several new properties of this dataset:


In [5]:
active_mnist.n_labelled

tensor(0)

In [6]:
active_mnist.n_unlabelled

tensor(60000)

We can also start labelling data. Either randomly, or based on specific indices:

In [7]:
active_mnist.label_randomly(10)
active_mnist.label([55, 56, 50100])

We've just labelled 10 points randomly, and 3 points based on specific indices.
Now, if we check how many have been labelled, we see that 13 have been labelled:


In [8]:
active_mnist.n_labelled

tensor(13)

In [9]:
active_mnist.n_unlabelled

tensor(59987)

We will also see that when we check the length of this dataset - something that
is done by e.g. pytorch `DataLoader` classes - it only gives the length of the
_labelled_ dataset:

In [12]:
len(active_mnist)

13

And, if we try to access an item, it will only allow us to index the _labelled_
datapoints:


In [13]:
active_mnist[0]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]), 8)

In [14]:
active_mnist[15]

IndexError: index 15 is out of bounds for dimension 0 with size 13

Instead, if we want to actually use the _unlabelled_ data, we need to use the
`pool` attribute of the active learning dataset, which is itself a dataset:


In [15]:
len(active_mnist.pool)

59987